In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc
import os
import random
import glob2
import tensorflow as tf
import keras
from skimage.io import imread
from PIL import Image
from glob import glob
from tensorflow.keras.preprocessing import image
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation,Conv2DTranspose,add,subtract
from keras.models import Model, Input 
from keras.models import load_model
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.optimizers import Adam
from keras import regularizers
import keras.backend as K
import h5py

Using TensorFlow backend.


In [2]:
#defeine the dataset
def DataSet():
    train_path_ori ='./Image/ori_train/'
    train_path_blr = './Image/blr_train/'
    
    test_path_ori ='./Image/ori_test/'
    test_path_blr = './Image/blr_test/'
    
    # os.listdir(path) 是 python 中的函数，它会列出 path 下的所有文件名
    # 比如说 imglist_train_ori 对象就包括了/train/ori/ 路径下所有的图片文件名
    imglist_train_ori = os.listdir(train_path_ori)
    imglist_train_blr = os.listdir(train_path_blr)
    
    # 下面两行代码读取了 /test/ori 和 /test/blr下的所有图片文件名
    imglist_test_ori = os.listdir(test_path_ori)
    imglist_test_blr = os.listdir(test_path_blr)
    
    
    ori_train = np.empty((len(imglist_train_ori) , 64, 64, 3))
    blr_train = np.empty((len(imglist_train_blr) , 64, 64, 3))
    
    # count 对象用来计数，每添加一张图片便加 1
    count = 0
    # 遍历 /train/ori 下所有图片，即训练集下所有的固体胶图片
    for img_name in imglist_train_ori:
        # 得到图片的路径
        img_path = train_path_ori + img_name
        # 通过 image.load_img() 函数读取对应的图片，并转换成目标大小
        #  image 是 tensorflow.keras.preprocessing 中的一个对象
        img = image.load_img(img_path, target_size=(64, 64))
        # 将图片转换成 numpy 数组，并除以 255 ，归一化
        # 转换之后 img 的 shape 是 （64，64，3）
        img = image.img_to_array(img) / 255.0
        # 将处理好的图片装进定义好的 ori_train 对象中
        ori_train[count] = img
        count+=1
        
    count = 0
    for img_name in imglist_train_blr:
        img_path = train_path_blr + img_name
        img = image.load_img(img_path, target_size=(64, 64))
        img = image.img_to_array(img) / 255.0
        blr_train[count] = img
        count+=1
        
        
    ori_test = np.empty((len(imglist_test_ori) , 64, 64, 3))
    blr_test = np.empty((len(imglist_test_blr) , 64, 64, 3)) 
    
    count = 0
    for img_name in imglist_test_ori:
        img_path = test_path_ori + img_name
        img = image.load_img(img_path, target_size=(64, 64))
        img = image.img_to_array(img) / 255.0
        ori_test[count] = img
        count+=1
        
    count = 0
    for img_name in imglist_test_blr:
        img_path = test_path_blr + img_name
        img = image.load_img(img_path, target_size=(64, 64))
        img = image.img_to_array(img) / 255.0
        blr_test[count] = img
        count+=1
        
  # 打乱训练集中的数据
    index = [i for i in range(len(ori_train))]
    random.shuffle(index)
    ori_train = ori_train[index]
    blr_train = blr_train[index]
    
    # 打乱测试集中的数据
    index = [i for i in range(len(ori_test))]
    random.shuffle(index)
    ori_test = ori_test[index]    
    blr_test = blr_test[index]	

    return ori_train,blr_train,ori_test,blr_test      

In [3]:
 #build the dataset
    ori_train,blr_train,ori_test,blr_test = DataSet()

In [2]:
#build up the CNN
deblur_CNN_input = Input(shape=(64,64,3))

Conv1 = Conv2D(filters=64, kernel_size=3, strides = 1, padding='same',dilation_rate=1)(deblur_CNN_input)
Conv1 = BatchNormalization()(Conv1)
Conv1 = Activation('relu')(Conv1)

Conv2 = Conv2D(filters=64, kernel_size=3, strides = 1, padding='same',dilation_rate=2)(Conv1)
Conv2 = BatchNormalization()(Conv2)
Conv2 = Activation('relu')(Conv2)

Conv3 = Conv2D(filters=64, kernel_size=3, strides = 1, padding='same',dilation_rate=3)(Conv2)
Conv3 = BatchNormalization()(Conv3)
Conv3 = Activation('relu')(Conv3)

Conv4 = Conv2D(filters=64, kernel_size=3, strides = 1, padding='same',dilation_rate=4)(Conv3)
Conv4 = BatchNormalization()(Conv4)
Convr4 = Activation('relu')(Conv4)

Conv5 = Conv2D(filters=64, kernel_size=3, strides = 1, padding='same',dilation_rate=3)(Conv4)
Conv5 = BatchNormalization()(Conv5)
Conv5 = Activation('relu')(Conv5)

Conv6 = Conv2D(filters=64, kernel_size=3, strides = 1, padding='same',dilation_rate=2)(Conv5)
Conv6 = BatchNormalization()(Conv6)
Conv6 = Activation('relu')(Conv6)

Deconv6=Conv2DTranspose(filters=64, kernel_size=3, strides = 1, padding='same',dilation_rate=2)(Conv6)
Deconv6= BatchNormalization()(Deconv6)
Deconv6 = Activation('relu')(Deconv6)

Deconv5=Conv2DTranspose(filters=64, kernel_size=3, strides = 1, padding='same',dilation_rate=3)(Deconv6)
Deconv5= BatchNormalization()(Deconv5)
skip3=add([Conv4,Deconv5 ])
Deconv5 = Activation('relu')(skip3)

Deconv4=Conv2DTranspose(filters=64, kernel_size=3, strides = 1, padding='same',dilation_rate=4)(Deconv5)
Deconv4= BatchNormalization()(Deconv4)
Deconv4 = Activation('relu')(Deconv4)

Deconv3=Conv2DTranspose(filters=64, kernel_size=3, strides = 1, padding='same',dilation_rate=3)(Deconv4)
Deconv3= BatchNormalization()(Deconv3)
skip2=add([Conv2,Deconv3 ])
Deconv3 = Activation('relu')(skip2)

Deconv2=Conv2DTranspose(filters=64, kernel_size=3, strides = 1, padding='same',dilation_rate=2)(Deconv3)
Deconv2= BatchNormalization()(Deconv2)
Deconv2 = Activation('relu')(Deconv2)

Deconv1=Conv2DTranspose(filters=3, kernel_size=3, strides = 1, padding='same',dilation_rate=1)(Deconv2)
skip1=add([deblur_CNN_input,Deconv1 ])
out = Activation('relu')(skip1)


deblur_CNN = Model(inputs= deblur_CNN_input, outputs=out)

In [3]:
deblur_CNN.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 64)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [16]:
#define smooth L1
HUBER_DELTA = 0.5
def smoothL1(y_true, y_pred):
   x  = K.abs(y_true - y_pred)
   x  = K.switch(x < HUBER_DELTA, 0.5 * x ** 2, HUBER_DELTA * (x - 0.5 * HUBER_DELTA))
   return  K.sum(x)

In [17]:
#set parameters
adam = Adam(lr= 0.001)

deblur_CNN.compile(optimizer= adam, loss= smoothL1,metrics=['accuracy'])
 #earlystop
earlystop = EarlyStopping(monitor='test_loss', patience=3, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=2, min_lr=0.000001)


In [18]:
deblur_CNN.load_weights('./MYCNN_with add_mae.h5')

In [8]:
#train my CNN
history=deblur_CNN.fit(blr_train, ori_train,
                epochs=15,
                batch_size=100,
                shuffle=True,
                validation_data=(blr_test, ori_test),
                callbacks=[reduce_lr]
       )
print(history.history.keys())

plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.show()

NameError: name 'blr_train' is not defined

In [10]:
deblur_CNN.evaluate(blr_test, ori_test, batch_size=32)

NameError: name 'blr_test' is not defined

In [25]:
#demo imgae
demo_path='./Image/demo/7_15/'
imglist_demo = os.listdir(demo_path)
demo = np.empty((len(imglist_demo) , 64, 64, 3))
count = 0
for img_name in imglist_demo:
    img_path = demo_path + img_name
    img = image.load_img(img_path, target_size=(64, 64))
    img = image.img_to_array(img) / 255.0
    demo[count] = img
    count+=1

In [26]:
def read_img(path):
    return tf.image.decode_image(tf.io.read_file(path))

In [27]:
#save image and calculate PSNR and SSIM
Deblurred = deblur_CNN.predict(demo)
i=0
index=1
Ptotal=0
Stotal=0
ori='E:/cjh/CNN/Image/demo_ori/'
save='E:/cjh/CNN/Image/demo/7_15/'
for i in range(5):
    im1 = Image.fromarray(( Deblurred[i]* 255).astype(np.uint8))
    im1.save(save+str(index)+'De.png')
    i=i+1
    index=index+1
    
    im1 =   read_img(save+str(index)+'.png')
    im2 =  read_img(ori+str(index)+'.png')
    # Compute PSNR over tf.uint8 Tensors.
    psnr = tf.image.psnr(im1, im2, max_val=255)
    ssim=tf.image.ssim(im1,im2,255)
    Ptotal=Ptotal+psnr
    Stotal=Stotal+ssim
   
    #print(psnr)
print(Ptotal/5)
print(Stotal/5)


In [44]:
deblur_CNN.save_weights('MYCNN_with add_mae.h5')